First, we need to read in our data from the world bank to establish our variables. From the website, I picked 19 different indicators from 2015 to 2020 initially with the expectation that many of them may not be complete or far reaching enough to be of much use.

In [ ]:
import pandas as pd
import numpy
from dfply import *
Worldbank_vars = pd.read_csv("./Worldbank_Data_format.csv")

#fix names

Worldbank_vars.columns = Worldbank_vars.columns.str.replace(' ', '_')

In [ ]:
Worldbank_vars

Winrar gave me some odd formating issues when exported directly from the compressed file. Resaving it as a .csv file after converting it from .xlsx seemed to eliminate it. I'm not quite sure what was causing it, but this resolved the issue.

With that taken care off, I will now inspect each variable and determine which ones appear to be the most viable for drawing conclusions. Mainly, I want to make sure more then a handul of countries are actually tracking these variables so that correlations across countires can be accomplished later down the line. My main year of focus will be 2019, as it is recent but represents the status quo before covid hit in early 2020. We have 217 countires that are kept track of, we want to try and keep as close to that as possible.

In [ ]:
(Worldbank_vars
>> filter_by(X["2019_[YR2019]"] == "..")
>> group_by(X.Series_Name)
>> summarise(na_counts = X.Series_Name.count() )

)

In [ ]:
#Worldbank_vars.groupby(["Series_Name"]).size()

It appears that much of this information is not complete. 2018 appears to have much more compiled based on the website output, perhaps wer should look there.


In [ ]:
(Worldbank_vars
>> filter_by(X["2018_[YR2018]"] == "..")
>> group_by(X.Series_Name)
>> summarise(na_counts = X.Series_Name.count() )

)

This looks much more complete on the whole, down from 9 to only 4 being entierly missing. Though its three years off, it seems to be the most recent we can get and still have meaningful amounts to work with.

The variables chosen from this are:

-Access to electricity (% of population)

-Air transport, freight (million ton-km)

-Air transport, passengers carried

-Automated teller machines (ATMs) (per 100,000 adults)

-Cost to import, border compliance (US$)

-Life expectancy at birth, total (years)

-Nurses and midwives (per 1,000 people)

-Population growth (annual %)	1

-Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)

-People practicing open defecation (% of population)

Though that last one does not have data from 2019 or 2018, it does have 2017 data as shown below. It is an interesting variable that I think is worth taking a look at and I am willing to go a bit farther back to grab it. Reasons I want to look into this identifiers along with the rest are discussed in detail in the essay.





In [ ]:
#only missing data from 21 countires in 2017

(Worldbank_vars
>> filter_by(X["2017_[YR2017]"] == "..")
>> group_by(X.Series_Name)
>> summarise(na_counts = X.Series_Name.count())
>> filter_by(X.Series_Name == "People practicing open defecation (% of population)")

)

Now that we have decided what we want, we need to drop what we don't. To do this explicitly, we will simply make a list of what we want first. I willu se the identifier codes as oppossed to the names as they play nicer with regex, are shorter to write, and should be more consistent over the long term

In [ ]:
Wanted_columns = ["EG.ELC.ACCS.ZS", "IS.AIR.GOOD.MT.K1", "IS.AIR.PSGR", "FB.ATM.TOTL.P5", "IC.IMP.CSBC.CD", "SP.DYN.LE00.IN", "SH.MED.NUMW.P3", "SP.POP.GROW", "SH.ALC.PCAP.LI","SH.STA.ODFC.ZS"]

In [ ]:

(Worldbank_vars
>> filter_by(X.Series_Code.str.contains('|'.join(Wanted_columns)) == True)
).head(10)

This works well, but now we want to get the variables in a form more conducive to looking at correlations. While correlations over time would be interesting, the pandemic iteself and useful data from the worldbank do not match up in a way that would allow us to do so. Instead, we will pull out specific variables from years and code them in columns on a per country basis. Then, we can pull in data for covid and join it on so that visuals and quickly be generated comparing countries various performance across these indeintifiers with covid metrics.

Since we have decided on using 2018's data for every variable save for one, we can start with that.

In [ ]:
(Worldbank_vars
>> drop(X["2015_[YR2015]"], X["2016_[YR2016]"], X["2019_[YR2019]"], X["2020_[YR2020]"])
>> group_by(X["Country_Name"])
>> spread(X["Series_Name"], X["2018_[YR2018]"])
>> filter_by(X["Country_Name"] == "United States")


).head(10)

In [ ]:
(Worldbank_vars
>> select(X["Country_Name"], X["Series_Name"], X["2018_[YR2018]"])
>> spread(X["Series_Name"], X["2018_[YR2018]"])


).head(10)

This works beautifully, but it is at this point that we probably want to change the null values in np.NaNs. We also will want to drop the defectaion variable we are using 2017 for.

In [ ]:
(Worldbank_vars
>> filter_by(X.Series_Code.str.contains('|'.join(Wanted_columns)) == True)
>> mutate(Year_2018 = X["2018_[YR2018]"].replace("..", np.NaN))
>> select(X["Country_Name"], X["Series_Name"], X["Year_2018"])
>> spread(X["Series_Name"], X["Year_2018"])
>> drop(X["People practicing open defecation (% of population)"])


).head(10)

Now, lets go back and grab that defecation column from 2017

In [ ]:

(Worldbank_vars
>> filter_by(X.Series_Code.str.contains("SH.STA.ODFC.ZS") == True)
>> mutate(Year_2017 = X["2017_[YR2017]"].replace("..", np.NaN))
>> select(X["Country_Name"], X["Series_Name"], X["Year_2017"])
>> spread(X["Series_Name"], X["Year_2017"])
>> select(X["Country_Name"], X["People practicing open defecation (% of population)"])
>> rename(People_practicing_open_defecation_percent_of_population_2017 = X["People practicing open defecation (% of population)"])

)

Now, we can join it onto what we had before

In [ ]:

Worlbank_2018_vars = (Worldbank_vars
>> filter_by(X.Series_Code.str.contains('|'.join(Wanted_columns)) == True)
>> mutate(Year_2018 = X["2018_[YR2018]"].replace("..", np.NaN))
>> select(X["Country_Name"], X["Series_Name"], X["Year_2018"])
>> spread(X["Series_Name"], X["Year_2018"])
>> drop(X["People practicing open defecation (% of population)"])
) 


Worlbank_2017_vars = (Worldbank_vars
>> filter_by(X.Series_Code.str.contains("SH.STA.ODFC.ZS") == True)
>> mutate(Year_2017 = X["2017_[YR2017]"].replace("..", np.NaN))
>> select(X["Country_Name"], X["Series_Name"], X["Year_2017"])
>> spread(X["Series_Name"], X["Year_2017"])
>> select(X["Country_Name"], X["People practicing open defecation (% of population)"])
>> rename(People_practicing_open_defecation_percent_of_population_2017 = X["People practicing open defecation (% of population)"])
)


#217 x 217 = 47,089, should be safe enough to throw inner joins out
Worlbank_recoded = Worlbank_2018_vars >> inner_join(Worlbank_2017_vars, by='Country_Name') 

Worlbank_recoded

In [ ]:
Worlbank_recoded.info()

We have not lost any rows while recoding, and every variable has at least 150 countries it maps to. The data types still need to be adjusted, but to be efficent I will wait until I have the covide data joined in to modify such.

Next, we need to bring in the covid data. I have three tables that follow the same general format of counting the number of deaths that occur cumulatively every day from 1/22/2020 up until 2/16/2021. To get the information marked in a way conducive to my current table, I will need to restandardize it by summing up the values across the dates.

The good news is that the same solution should work for all three tables, so I will implment a solution for one of them and then extend that process to the other two.



In [ ]:
Covid_Deaths = pd.read_csv("./deaths.csv")

Covid_Deaths

In [ ]:
Covid_Deaths.info()

Given that the information is cumulative, we can simply chop off every column that isn't that last one, and relabel it as "Covid_Deaths_as_of_2_16_2021". Many countries appear to not be keeping track of this data, nor any covid data for that matter. Such countries will be eliminated from the final file once all is said in done so as to not add bloat to our analysis.

In [ ]:
(Covid_Deaths
>> select(X["Country"], X["2/16/2021"])
>> rename(Covid_Deaths_as_of_2_16_2021 = X["2/16/2021"])


).info()

Time series tend to suffer from weird misalingment among when tracking multiple variables, at least in my experience. For that reason, I am going to quickly check the last columns to make sure data is not simply being dropped prematuerly for some countries.  

In [ ]:
(Covid_Deaths
>> select(X["Country"], X["1/22/2020"], X["2/16/2021"])
>> filter_by(X["2/16/2021"] == 0)


)

The first and last series appear to match up in terms of being zero. Therefore, it appears that either this countries wethered covid perfectly or are not tracking/being tracked. When we get to culling data items further down, this will be considered. The good news now is that we can confirm that we aren't missing anything not so disparged it would not be not worth looking at anyway.

Next, we need to ensure that our country names line up with each other for the join. A trivial check makes it seem as though it should work fine, but a few cases in the were parenthesisa are involved could throw a wrench in. Fortunatley, 217 is not a number incappable of being looked over quickly to ensure perfect compliance. But before we muse about a solution,we have to confirm a problem.

In [ ]:
Worlbank_Names = (Worlbank_recoded >> select(X["Country_Name"]))

Covid_Names = (Covid_Deaths >> select(X["Country"]) >> rename(Country_Name = X["Country"]))


Covid_Names >> inner_join(Worlbank_Names, by='Country_Name')
   


173 is a far cry from 216. Lets take a look at the outer join to see the disparities.

In [ ]:
Covid_Names = (Covid_Deaths >> select(X["Country"]) )


Joined_Names = Covid_Names.merge(Worlbank_Names, how="outer", left_on='Country', right_on='Country_Name')

Joined_Names

Now, I will inspect where either column has a null to try and see the discprepancies.

In [ ]:
Problem_Names = (Joined_Names >> filter_by((X["Country"].isnull() == True) | (X["Country_Name"].isnull() == True)))

Problem_Names.info()

We appear to have 88 mismatches, leading to hopefully 44 misalingments. Casing does not seem to be an issue, but rather the usage of true vs. common titles and abbreviations. Lets look and try to evaluate cases to see if any pattersn appear or if manually recoding is needed.

In [ ]:
Problem_Names.iloc[0:50]

In [ ]:
Problem_Names.iloc[50:88]

44 Problems

1. Russia vs Russian Federation
2. Iran vs Iran, Islamic Rep.
3. Slovakia vs Slovak Republic
4. Egypt vs Egypt, Arab Rep.
5. Burma vs Myanmar
6. Venezuela vs Venezuela, RB
7. Macedonia vs North Macedonia	
8. Kyrgyzstan vs Kyrgyz Republic
9. South Korea vs Korea, Rep.
10. Ivory Coast vs Cote d'Ivoire
11. Congo (Kinshasa) vs Congo, Dem. Rep.
12. Syria vs Syrian Arab Republic
13. Cape Verde vs Cabo Verde
14. Congo (Brazzaville)	vs Congo, Rep.
15. Bahamas	vs Bahamas, The
16. The Gambia vs Gambia, The
17. Saint Lucia	vs St. Lucia
18. Yemen vs Yemen, Rep.
19. Saint Vincent and the Grenadines vs St. Vincent and the Grenadines
20. Taiwan vs https://datahelpdesk.worldbank.org/knowledgebase/articles/114933-where-are-your-data-on-taiwan
21. Cruise Ship vs None
22. Diamond Princess vs None
23. Others vs None
24. Brunei vs Brunei Darussalam
25. Hong Kong vs Hong Kong SAR, China
26. East Timor vs Timor-Leste
27. Guadeloupe vs None
28. Laos vs Lao PDR
29. Reunion vs None
30. Saint Kitts and Nevis vs St. Kitts and Nevis 
31. Martinique vs None
32. Vatican City vs None
33. French Guiana vs None
34. Macau vs Macao SAR, China
35. MS Zaandam vs None
36. Mayotte	vs None
37. St. Martin vs St. Martin (French part)
38. Micronesia vs Micronesia, Fed. Sts.
39. North Ireland vs None
40. Saint Barthelemy vs None
41. Congo vs Congo, Dem. Rep.
42. Guernsey vs None
43. Jersey vs None
44. Palestine vs None

From this list, most can simply be recoded, the rest must be handled seperatley.

Cruise Ships:

Cruise ships are not countires, and will simply be removed from the analysis. Though individuals on it may hail from other countries, they do not follow the same soverign law. This analysis is concerned with cases in areas, not from areas.

    1.Cruise Ship vs None
    2.Diamond Princess vs None
    3.MS Zaandam vs None

Other:

Metadata on this category is vague. From what I understand, it is a collection of different, geographically distinct areas of small or undefined size. Because it lacks a unified identity, It cannot be joined to worldbank identifiers and thus unfortunatley must be removed.

    1.Others vs None

French Departments and territories:

French Departmens in the Carribean and South American each have there own, usually complex relationship with France itself. However, by and large, they tend to follow French federal law and policy. Though seperated in the covid data for sake of geographical tracking, as far as the worldbank is concerned, France is France. For the sake of our analysis, these will in turn also simply be considered as "France".

    1.Guadeloupe vs None
    2.Reunion vs None
    3.Martinique vs None
    4.French Guiana vs None
    5.Mayotte vs None
    6.Saint Barthelemy

Crown Dependencies:

Similar to France, the UK has many similar former imperial territories. However, they tend to have a bit more self-governance than most french ones by comparison. However, they are still basically the UK and mirror it well. They also are much closer to the british isles then when compared to France and its globe spanning former colonies.

    1.Guernsey vs None
    2.Jersey vs None
    

Worldbank has no data:

Vatican city is its own beast when it comes to everything and has a true permanent population of about 500. The Worldbank has little information on this area, yet it is different enough in policy from Italy that it seems unfair enough to include it within it. Therefore it shall be removed.

To side-step the entire issue, Palestine is going to be combined with West bank and Gaza, which goegraphically lines up mostly with where palenstinians live.

    1.Vatican City vs None
    2.Palestine vs None

To be combined with UK:

I am not about to get into a political discussion about the troubles, but economically speaking, Northern Ireland is considered by the world bank to be "UK" and therefore is a part of their demographics.

    1.North Ireland vs None

The Congo has three sections in the covid data, Kinshasa, Brazzaville, and "Congo". I will combine Kinshasa with "Congo," as that city is located within the larger and UN backed PRC, wheras the smaller area of Brazzavile is seperate.

Taiwan data is stated to not be included with China, but it also has no category for World development indicators on its own. I will add it to the data once World Bank grows a pair and adds it to the country list where it belongs.

In the end, that leaves us with 12 categories to recode, so our output should have 205 as a result.


Now, these recodings must be done as efficently as possible. We will stick with the Worldbank names, as they tend to follow the actual country names as opposed to the shorthands sometimes used. This does mean a bit more manually recoding, but with cases like the congo and the balkan states being much more clear than the shorthand usage. It is also polite.

In [ ]:
Covid_Names

In [ ]:
Covid_Names_Adj = (Covid_Names
>> mutate(Country2 = X["Country"]) #fix the value error
>> mutate(Country2 = case_when(
    
    #French Departments
    (X["Country2"].eq("Guadeloupe") == True, "France"),
    (X["Country2"].eq("Reunion") == True, "France"),
    (X["Country2"].eq("Martinique") == True, "France"),
    (X["Country2"].eq("French Guiana") == True, "France"),
    (X["Country2"].eq("Mayotte") == True, "France"),
    (X["Country2"].eq("Saint Barthelemy") == True, "France"),
    (X["Country2"].eq("St. Martin") == True, "France"),


    #replace all "Saint" with St. (done after to deal with French departmens with sains)
    (X["Country2"].str.contains("Saint") == True, X["Country2"].str.replace("Saint","St.")),


    #manually recode countries
    (X["Country2"].eq("Russia") == True, "Russian Federation"),
    (X["Country2"].eq("Iran") == True, "Iran, Islamic Rep."),
    (X["Country2"].eq("Slovakia") == True, "Slovak Republic"),
    (X["Country2"].eq("Venezuela") == True, "Venezuela, RB"),
    (X["Country2"].eq("Egypt") == True, "Egypt, Arab Rep."),
    (X["Country2"].eq("Burma") == True, "Myanmar"),
    (X["Country2"].eq("Macedonia") == True, "North Macedonia"),
    (X["Country2"].eq("Kyrgyzstan") == True, "Kyrgyz Republic"),
    (X["Country2"].eq("South Korea") == True, "Korea, Rep."),
    (X["Country2"].eq("Ivory Coast") == True, "Cote d'Ivoire"),
    (X["Country2"].eq("Bahamas") == True, "Bahamas, The"),
    (X["Country2"].eq("The Gambia") == True, "Gambia, The"),
    (X["Country2"].eq("Yemen") == True, "Yemen, Rep."),
    (X["Country2"].eq("Brunei") == True, "Brunei Darussalam"),
    (X["Country2"].eq("Hong Kong") == True, "Hong Kong SAR, China"),
    (X["Country2"].eq("East Timor") == True, "Timor-Leste"),
    (X["Country2"].eq("Laos") == True, "Lao PDR"),
    (X["Country2"].eq("Macau") == True, "Macao SAR, China"),
    (X["Country2"].eq("Micronesia") == True, "Micronesia, Fed. Sts."),

    (X["Country2"].eq("Syria") == True, "Syrian Arab Republic"),
    (X["Country2"].eq("Cape Verde") == True, "Cabo Verde"),


    #Crown Dependencies and Northern Ireland
    (X["Country2"].eq("Guernsey") == True, "United Kingdom"),
    (X["Country2"].eq("Jersey") == True, "United Kingdom"),
    (X["Country2"].eq("North Ireland") == True, "United Kingdom"),


    #Sorting out Congo
    (X["Country2"].eq("Congo") == True, "Congo, Dem. Rep."),
    (X["Country2"].eq("Congo (Kinshasa)") == True, "Congo, Dem. Rep."),
    (X["Country2"].eq("Congo (Brazzaville)") == True, "Congo, Rep."),


    #Palestine to west bank and gaza
    (X["Country2"].eq("Palestine") == True, "West Bank and Gaza"),

    (True, X["Country"])

))

>> select(X["Country2"])
>> rename(Country = X["Country2"])

) 


Recoded_Names = Covid_Names_Adj.merge(Worlbank_Names, how="outer", left_on='Country', right_on='Country_Name')

Recoded_Names

Now, we need to make sure every Covid Country has a mapping to some WorlBank data if it is not one we are removing.

In [ ]:
Check_Rec_Names = (Recoded_Names >> filter_by((X["Country_Name"].isnull() == True)))

Check_Rec_Names

The above rows will be dropped when we inner join, but that is fine as we do not want them anyway. Now, I will do again with the data and sum together rows representing the same country.

In [ ]:
Covid_Names_Adj["Country"].drop_duplicates()

In [ ]:
(Covid_Deaths
>> select(X["Country"], X["2/16/2021"])
>> rename(Covid_Deaths_as_of_2_16_2021 = X["2/16/2021"])

>> mutate(Country2 = X["Country"]) #fix the value error
>> mutate(Country2 = case_when(
    
    #French Departments
    (X["Country2"].eq("Guadeloupe") == True, "France"),
    (X["Country2"].eq("Reunion") == True, "France"),
    (X["Country2"].eq("Martinique") == True, "France"),
    (X["Country2"].eq("French Guiana") == True, "France"),
    (X["Country2"].eq("Mayotte") == True, "France"),
    (X["Country2"].eq("Saint Barthelemy") == True, "France"),
    (X["Country2"].eq("St. Martin") == True, "France"),


    #replace all "Saint" with St. (done after to deal with French departmens with sains)
    (X["Country2"].str.contains("Saint") == True, X["Country2"].str.replace("Saint","St.")),


    #manually recode countries
    (X["Country2"].eq("Russia") == True, "Russian Federation"),
    (X["Country2"].eq("Iran") == True, "Iran, Islamic Rep."),
    (X["Country2"].eq("Slovakia") == True, "Slovak Republic"),
    (X["Country2"].eq("Venezuela") == True, "Venezuela, RB"),
    (X["Country2"].eq("Egypt") == True, "Egypt, Arab Rep."),
    (X["Country2"].eq("Burma") == True, "Myanmar"),
    (X["Country2"].eq("Macedonia") == True, "North Macedonia"),
    (X["Country2"].eq("Kyrgyzstan") == True, "Kyrgyz Republic"),
    (X["Country2"].eq("South Korea") == True, "Korea, Rep."),
    (X["Country2"].eq("Ivory Coast") == True, "Cote d'Ivoire"),
    (X["Country2"].eq("Bahamas") == True, "Bahamas, The"),
    (X["Country2"].eq("The Gambia") == True, "Gambia, The"),
    (X["Country2"].eq("Yemen") == True, "Yemen, Rep."),
    (X["Country2"].eq("Brunei") == True, "Brunei Darussalam"),
    (X["Country2"].eq("Hong Kong") == True, "Hong Kong SAR, China"),
    (X["Country2"].eq("East Timor") == True, "Timor-Leste"),
    (X["Country2"].eq("Laos") == True, "Lao PDR"),
    (X["Country2"].eq("Macau") == True, "Macao SAR, China"),
    (X["Country2"].eq("Micronesia") == True, "Micronesia, Fed. Sts."),

    (X["Country2"].eq("Syria") == True, "Syrian Arab Republic"),
    (X["Country2"].eq("Cape Verde") == True, "Cabo Verde"),


    #Crown Dependencies and Northern Ireland
    (X["Country2"].eq("Guernsey") == True, "United Kingdom"),
    (X["Country2"].eq("Jersey") == True, "United Kingdom"),
    (X["Country2"].eq("North Ireland") == True, "United Kingdom"),


    #Sorting out Congo
    (X["Country2"].eq("Congo") == True, "Congo, Dem. Rep."),
    (X["Country2"].eq("Congo (Kinshasa)") == True, "Congo, Dem. Rep."),
    (X["Country2"].eq("Congo (Brazzaville)") == True, "Congo, Rep."),


    #Palestine to west bank and gaza
    (X["Country2"].eq("Palestine") == True, "West Bank and Gaza"),

    (True, X["Country"])

))

>> select(X["Country2"], X["Covid_Deaths_as_of_2_16_2021"])
>> rename(Country = X["Country2"])


>> group_by(X["Country"])
>> summarise(Covid_Deaths_as_of_2_16_2021 = X["Covid_Deaths_as_of_2_16_2021"].sum())


)

We are now down to 205 rows, which is the right number given the 12 we recoded into others. Now, we need to join it onto the worldbank data. We will lose rows doing this, but the ones that we do will be ones with either no covid data or were deemed not relevant to our coming analysis. Since we already checked for name compliance above, we do not need to do it again.

In [ ]:
Covid_Deaths_Cleaned = (Covid_Deaths
>> select(X["Country"], X["2/16/2021"])
>> rename(Covid_Deaths_as_of_2_16_2021 = X["2/16/2021"])

>> mutate(Country2 = X["Country"]) #fix the value error
>> mutate(Country2 = case_when(
    
    #French Departments
    (X["Country2"].eq("Guadeloupe") == True, "France"),
    (X["Country2"].eq("Reunion") == True, "France"),
    (X["Country2"].eq("Martinique") == True, "France"),
    (X["Country2"].eq("French Guiana") == True, "France"),
    (X["Country2"].eq("Mayotte") == True, "France"),
    (X["Country2"].eq("Saint Barthelemy") == True, "France"),
    (X["Country2"].eq("St. Martin") == True, "France"),


    #replace all "Saint" with St. (done after to deal with French departmens with sains)
    (X["Country2"].str.contains("Saint") == True, X["Country2"].str.replace("Saint","St.")),


    #manually recode countries
    (X["Country2"].eq("Russia") == True, "Russian Federation"),
    (X["Country2"].eq("Iran") == True, "Iran, Islamic Rep."),
    (X["Country2"].eq("Slovakia") == True, "Slovak Republic"),
    (X["Country2"].eq("Venezuela") == True, "Venezuela, RB"),
    (X["Country2"].eq("Egypt") == True, "Egypt, Arab Rep."),
    (X["Country2"].eq("Burma") == True, "Myanmar"),
    (X["Country2"].eq("Macedonia") == True, "North Macedonia"),
    (X["Country2"].eq("Kyrgyzstan") == True, "Kyrgyz Republic"),
    (X["Country2"].eq("South Korea") == True, "Korea, Rep."),
    (X["Country2"].eq("Ivory Coast") == True, "Cote d'Ivoire"),
    (X["Country2"].eq("Bahamas") == True, "Bahamas, The"),
    (X["Country2"].eq("The Gambia") == True, "Gambia, The"),
    (X["Country2"].eq("Yemen") == True, "Yemen, Rep."),
    (X["Country2"].eq("Brunei") == True, "Brunei Darussalam"),
    (X["Country2"].eq("Hong Kong") == True, "Hong Kong SAR, China"),
    (X["Country2"].eq("East Timor") == True, "Timor-Leste"),
    (X["Country2"].eq("Laos") == True, "Lao PDR"),
    (X["Country2"].eq("Macau") == True, "Macao SAR, China"),
    (X["Country2"].eq("Micronesia") == True, "Micronesia, Fed. Sts."),

    (X["Country2"].eq("Syria") == True, "Syrian Arab Republic"),
    (X["Country2"].eq("Cape Verde") == True, "Cabo Verde"),


    #Crown Dependencies and Northern Ireland
    (X["Country2"].eq("Guernsey") == True, "United Kingdom"),
    (X["Country2"].eq("Jersey") == True, "United Kingdom"),
    (X["Country2"].eq("North Ireland") == True, "United Kingdom"),


    #Sorting out Congo
    (X["Country2"].eq("Congo") == True, "Congo, Dem. Rep."),
    (X["Country2"].eq("Congo (Kinshasa)") == True, "Congo, Dem. Rep."),
    (X["Country2"].eq("Congo (Brazzaville)") == True, "Congo, Rep."),


    #Palestine to west bank and gaza
    (X["Country2"].eq("Palestine") == True, "West Bank and Gaza"),

    (True, X["Country"])

))

>> select(X["Country2"], X["Covid_Deaths_as_of_2_16_2021"])
>> rename(Country_Name = X["Country2"])


>> group_by(X["Country_Name"])
>> summarise(Covid_Deaths_as_of_2_16_2021 = X["Covid_Deaths_as_of_2_16_2021"].sum())


) 


Worlbank_2018_vars = (Worldbank_vars
>> filter_by(X.Series_Code.str.contains('|'.join(Wanted_columns)) == True)
>> mutate(Year_2018 = X["2018_[YR2018]"].replace("..", np.NaN))
>> select(X["Country_Name"], X["Series_Name"], X["Year_2018"])
>> spread(X["Series_Name"], X["Year_2018"])
>> drop(X["People practicing open defecation (% of population)"])
) 


Worlbank_2017_vars = (Worldbank_vars
>> filter_by(X.Series_Code.str.contains("SH.STA.ODFC.ZS") == True)
>> mutate(Year_2017 = X["2017_[YR2017]"].replace("..", np.NaN))
>> select(X["Country_Name"], X["Series_Name"], X["Year_2017"])
>> spread(X["Series_Name"], X["Year_2017"])
>> select(X["Country_Name"], X["People practicing open defecation (% of population)"])
>> rename(People_practicing_open_defecation_percent_of_population_2017 = X["People practicing open defecation (% of population)"])
)



 
Output = (Worlbank_2018_vars 
>> inner_join(Worlbank_2017_vars, by="Country_Name") 
>> inner_join(Covid_Deaths_Cleaned, by="Country_Name")
)




This solution works out well for Covid deaths, but we still want to grab recoveries and cases. If all goes well, A two joins in the first pipe should be all that is needed, but we still need to inspect them for problems before going in blind.

A quick glance at the recovery file already spills the beans on its main issue in that countries sometimes don't accumulate like they are supposed to. That is exceptionally annoying. Fortunatley, because the data is cumulative, we only need to grab the larget data item in each row, which we can iterate with a simple list comprehension.

In [ ]:
Covid_Recoveries = pd.read_csv("./recoveries.csv")

Covid_Recoveries

In [ ]:
Covid_Recoveries_fixed = Covid_Recoveries.max(axis=1).to_frame() >> rename(Covid_Recoveries_as_of_2_16_2021 = X[0])

Covid_Recoveries_fixed

Cases looks fine, but we will do the same just to be safe.

In [ ]:
Covid_Cases = pd.read_csv("./cases.csv")

Covid_Cases

In [ ]:
Covid_Cases_fixed = Covid_Cases.max(axis=1).to_frame() >> rename(Covid_Cases_as_of_2_16_2021 = X[0])

Covid_Cases_fixed

In [ ]:
Covid_data_combined = pd.concat([Covid_Deaths, Covid_Cases_fixed, Covid_Recoveries_fixed], axis=1)

Covid_data_combined

Now, we will join these onto our main output to finish up our data formation.

In [ ]:
Covid_Data_Cleaned = (Covid_data_combined
>> select(X["Country"], X["2/16/2021"], X["Covid_Cases_as_of_2_16_2021"], X["Covid_Recoveries_as_of_2_16_2021"])
>> rename(Covid_Deaths_as_of_2_16_2021 = X["2/16/2021"])
>> mutate(Country2 = X["Country"]) #fix the value error
>> mutate(Country2 = case_when(
    
    #French Departments
    (X["Country2"].eq("Guadeloupe") == True, "France"),
    (X["Country2"].eq("Reunion") == True, "France"),
    (X["Country2"].eq("Martinique") == True, "France"),
    (X["Country2"].eq("French Guiana") == True, "France"),
    (X["Country2"].eq("Mayotte") == True, "France"),
    (X["Country2"].eq("Saint Barthelemy") == True, "France"),
    (X["Country2"].eq("St. Martin") == True, "France"),


    #replace all "Saint" with St. (done after to deal with French departmens with sains)
    (X["Country2"].str.contains("Saint") == True, X["Country2"].str.replace("Saint","St.")),


    #manually recode countries
    (X["Country2"].eq("Russia") == True, "Russian Federation"),
    (X["Country2"].eq("Iran") == True, "Iran, Islamic Rep."),
    (X["Country2"].eq("Slovakia") == True, "Slovak Republic"),
    (X["Country2"].eq("Venezuela") == True, "Venezuela, RB"),
    (X["Country2"].eq("Egypt") == True, "Egypt, Arab Rep."),
    (X["Country2"].eq("Burma") == True, "Myanmar"),
    (X["Country2"].eq("Macedonia") == True, "North Macedonia"),
    (X["Country2"].eq("Kyrgyzstan") == True, "Kyrgyz Republic"),
    (X["Country2"].eq("South Korea") == True, "Korea, Rep."),
    (X["Country2"].eq("Ivory Coast") == True, "Cote d'Ivoire"),
    (X["Country2"].eq("Bahamas") == True, "Bahamas, The"),
    (X["Country2"].eq("The Gambia") == True, "Gambia, The"),
    (X["Country2"].eq("Yemen") == True, "Yemen, Rep."),
    (X["Country2"].eq("Brunei") == True, "Brunei Darussalam"),
    (X["Country2"].eq("Hong Kong") == True, "Hong Kong SAR, China"),
    (X["Country2"].eq("East Timor") == True, "Timor-Leste"),
    (X["Country2"].eq("Laos") == True, "Lao PDR"),
    (X["Country2"].eq("Macau") == True, "Macao SAR, China"),
    (X["Country2"].eq("Micronesia") == True, "Micronesia, Fed. Sts."),
    (X["Country2"].eq("Syria") == True, "Syrian Arab Republic"),
    (X["Country2"].eq("Cape Verde") == True, "Cabo Verde"),


    #Crown Dependencies and Northern Ireland
    (X["Country2"].eq("Guernsey") == True, "United Kingdom"),
    (X["Country2"].eq("Jersey") == True, "United Kingdom"),
    (X["Country2"].eq("North Ireland") == True, "United Kingdom"),


    #Sorting out Congo
    (X["Country2"].eq("Congo") == True, "Congo, Dem. Rep."),
    (X["Country2"].eq("Congo (Kinshasa)") == True, "Congo, Dem. Rep."),
    (X["Country2"].eq("Congo (Brazzaville)") == True, "Congo, Rep."),


    #Palestine to west bank and gaza
    (X["Country2"].eq("Palestine") == True, "West Bank and Gaza"),

    (True, X["Country"])

))

>> select(X["Country2"], X["Covid_Deaths_as_of_2_16_2021"], X["Covid_Cases_as_of_2_16_2021"], X["Covid_Recoveries_as_of_2_16_2021"])

>> rename(Country_Name = X["Country2"])


>> group_by(X["Country_Name"])
>> summarise(Covid_Deaths_as_of_2_16_2021 = X["Covid_Deaths_as_of_2_16_2021"].sum(), Covid_Cases_as_of_2_16_2021 = X["Covid_Cases_as_of_2_16_2021"].sum(), Covid_Recoviers_as_of_2_16_2021 = X["Covid_Recoveries_as_of_2_16_2021"].sum()  )


) 

Covid_Data_Cleaned

In [ ]:
Worlbank_2018_vars = (Worldbank_vars
>> filter_by(X.Series_Code.str.contains('|'.join(Wanted_columns)) == True)
>> mutate(Year_2018 = X["2018_[YR2018]"].replace("..", np.NaN))
>> select(X["Country_Name"], X["Series_Name"], X["Year_2018"])
>> spread(X["Series_Name"], X["Year_2018"])
>> drop(X["People practicing open defecation (% of population)"])
) 


Worlbank_2017_vars = (Worldbank_vars
>> filter_by(X.Series_Code.str.contains("SH.STA.ODFC.ZS") == True)
>> mutate(Year_2017 = X["2017_[YR2017]"].replace("..", np.NaN))
>> select(X["Country_Name"], X["Series_Name"], X["Year_2017"])
>> spread(X["Series_Name"], X["Year_2017"])
>> select(X["Country_Name"], X["People practicing open defecation (% of population)"])
>> rename(People_practicing_open_defecation_percent_of_population_2017 = X["People practicing open defecation (% of population)"])
)



Output = (Worlbank_2018_vars 
>> inner_join(Worlbank_2017_vars, by="Country_Name") 
>> inner_join(Covid_Data_Cleaned, by="Country_Name")
)

Output


Now that we have our output, it is time to bring the whole process in an organized fashion together and optimize it further. Ideally, one could run just the following from start to finish as a better version of the process engineered above.

In [2]:
#packages

import pandas as pd
import numpy
from dfply import *

In [3]:
# Read in files

Worldbank_vars = pd.read_csv("./Worldbank_Data_format.csv")
Covid_Recoveries = pd.read_csv("./recoveries.csv")
Covid_Cases = pd.read_csv("./cases.csv")
Covid_Deaths = pd.read_csv("./deaths.csv")


In [4]:
#use our trick to get the maximum accumulated, future proofing it by using the columns of each file's country names unless the order ever changes across each for any reason. It may be arguably Wet but because we are relying on different source files, focusing operations on each one could allow for easier maintenance down the line should one of them require it.


#Recoveries
Covid_Recoveries_Total= Covid_Recoveries >> select(["Country"])
Covid_Recoveries_Total["Covid_Recoveries_as_of_2_16_2021"] = Covid_Recoveries.max(axis=1)


#Cases
Covid_Cases_Total= Covid_Cases >> select(["Country"])
Covid_Cases_Total["Covid_Cases_as_of_2_16_2021"] = Covid_Cases.max(axis=1)


#Deaths
Covid_Deaths_Total= Covid_Deaths >> select(["Country"])
Covid_Deaths_Total["Covid_Deaths_as_of_2_16_2021"] = Covid_Deaths.max(axis=1)


Covid_Data_Combined = (Covid_Recoveries_Total 
>> inner_join(Covid_Cases_Total, by="Country") 
>> inner_join(Covid_Deaths_Total, by="Country")
)

Covid_Data_Combined

,Country,Covid_Recoveries_as_of_2_16_2021,Covid_Cases_as_of_2_16_2021,Covid_Deaths_as_of_2_16_2021
0,United States,6399531,27756624,488081
1,India,10644858,10937320,155913
2,Brazil,8847264,9921981,240940
3,United Kingdom,11022,4070332,118421
4,Russia,3584477,4053535,79659
...,...,...,...,...
212,Guam,0,3,0
213,Guernsey,0,1,0
214,Jersey,0,2,0
215,Palestine,0,25,0


In [5]:
Covid_Data_Cleaned = (Covid_Data_Combined
>> mutate(Country2 = X["Country"]) #fix the value error
>> mutate(Country2 = case_when(
    
    #French Departments
    (X["Country2"].eq("Guadeloupe") == True, "France"),
    (X["Country2"].eq("Reunion") == True, "France"),
    (X["Country2"].eq("Martinique") == True, "France"),
    (X["Country2"].eq("French Guiana") == True, "France"),
    (X["Country2"].eq("Mayotte") == True, "France"),
    (X["Country2"].eq("Saint Barthelemy") == True, "France"),
    (X["Country2"].eq("St. Martin") == True, "France"),


    #replace all "Saint" with St. (done after to deal with French departmens with sains)
    (X["Country2"].str.contains("Saint") == True, X["Country2"].str.replace("Saint","St.")),


    #manually recode countries
    (X["Country2"].eq("Russia") == True, "Russian Federation"),
    (X["Country2"].eq("Iran") == True, "Iran, Islamic Rep."),
    (X["Country2"].eq("Slovakia") == True, "Slovak Republic"),
    (X["Country2"].eq("Venezuela") == True, "Venezuela, RB"),
    (X["Country2"].eq("Egypt") == True, "Egypt, Arab Rep."),
    (X["Country2"].eq("Burma") == True, "Myanmar"),
    (X["Country2"].eq("Macedonia") == True, "North Macedonia"),
    (X["Country2"].eq("Kyrgyzstan") == True, "Kyrgyz Republic"),
    (X["Country2"].eq("South Korea") == True, "Korea, Rep."),
    (X["Country2"].eq("Ivory Coast") == True, "Cote d'Ivoire"),
    (X["Country2"].eq("Bahamas") == True, "Bahamas, The"),
    (X["Country2"].eq("The Gambia") == True, "Gambia, The"),
    (X["Country2"].eq("Yemen") == True, "Yemen, Rep."),
    (X["Country2"].eq("Brunei") == True, "Brunei Darussalam"),
    (X["Country2"].eq("Hong Kong") == True, "Hong Kong SAR, China"),
    (X["Country2"].eq("East Timor") == True, "Timor-Leste"),
    (X["Country2"].eq("Laos") == True, "Lao PDR"),
    (X["Country2"].eq("Macau") == True, "Macao SAR, China"),
    (X["Country2"].eq("Micronesia") == True, "Micronesia, Fed. Sts."),
    (X["Country2"].eq("Syria") == True, "Syrian Arab Republic"),
    (X["Country2"].eq("Cape Verde") == True, "Cabo Verde"),


    #Crown Dependencies and Northern Ireland
    (X["Country2"].eq("Guernsey") == True, "United Kingdom"),
    (X["Country2"].eq("Jersey") == True, "United Kingdom"),
    (X["Country2"].eq("North Ireland") == True, "United Kingdom"),


    #Sorting out Congo
    (X["Country2"].eq("Congo") == True, "Congo, Dem. Rep."),
    (X["Country2"].eq("Congo (Kinshasa)") == True, "Congo, Dem. Rep."),
    (X["Country2"].eq("Congo (Brazzaville)") == True, "Congo, Rep."),


    #Palestine to west bank and gaza
    (X["Country2"].eq("Palestine") == True, "West Bank and Gaza"),

    (True, X["Country"])

))

>> select(X["Country2"], X["Covid_Deaths_as_of_2_16_2021"], X["Covid_Cases_as_of_2_16_2021"], X["Covid_Recoveries_as_of_2_16_2021"])

>> rename(Country_Name = X["Country2"])


>> group_by(X["Country_Name"])
>> summarise(Covid_Deaths_as_of_2_16_2021 = X["Covid_Deaths_as_of_2_16_2021"].sum(), Covid_Cases_as_of_2_16_2021 = X["Covid_Cases_as_of_2_16_2021"].sum(), Covid_Recoviers_as_of_2_16_2021 = X["Covid_Recoveries_as_of_2_16_2021"].sum()  )


) 

/home/sandrews/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages/dfply/vector.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outcome[~logical] = np.nan
/home/sandrews/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages/dfply/vector.py:110: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  return coalescer.lookup(np.arange(coalescer.shape[0]), min_nonna)


In [7]:
#now we get the worldbank data form 2018 and 217 in the forms we want. This is done in two seperate pipes to keeps things more readable and easy to understand

#fix names first
Worldbank_vars.columns = Worldbank_vars.columns.str.replace(' ', '_')

Wanted_2018_columns = ["EG.ELC.ACCS.ZS", "IS.AIR.GOOD.MT.K1", "IS.AIR.PSGR", "FB.ATM.TOTL.P5", "IC.IMP.CSBC.CD", "SP.DYN.LE00.IN", "SH.MED.NUMW.P3", "SP.POP.GROW", "SH.ALC.PCAP.LI","SH.STA.ODFC.ZS"]

Worlbank_2018_vars = (Worldbank_vars
>> filter_by(X.Series_Code.str.contains('|'.join(Wanted_2018_columns)) == True)
>> mutate(Year_2018 = X["2018_[YR2018]"].replace("..", np.NaN))
>> select(X["Country_Name"], X["Series_Name"], X["Year_2018"])
>> spread(X["Series_Name"], X["Year_2018"])
>> drop(X["People practicing open defecation (% of population)"])
) 


Worlbank_2017_vars = (Worldbank_vars
>> filter_by(X.Series_Code.str.contains("SH.STA.ODFC.ZS") == True)
>> mutate(Year_2017 = X["2017_[YR2017]"].replace("..", np.NaN))
>> select(X["Country_Name"], X["Series_Name"], X["Year_2017"])
>> spread(X["Series_Name"], X["Year_2017"])
>> select(X["Country_Name"], X["People practicing open defecation (% of population)"])
>> rename(People_practicing_open_defecation_percent_of_population_2017 = X["People practicing open defecation (% of population)"])
)

In [37]:


World_Bank_Covid_Data_Combined = (Worlbank_2018_vars 
>> inner_join(Worlbank_2017_vars, by="Country_Name") 
>> inner_join(Covid_Data_Cleaned, by="Country_Name")
)

World_Bank_Covid_Data_Combined


,Country_Name,Access to electricity (% of population),"Air transport, freight (million ton-km)","Air transport, passengers carried","Automated teller machines (ATMs) (per 100,000 adults)","Cost to import, border compliance (US$)","Life expectancy at birth, total (years)","Nurses and midwives (per 1,000 people)",Population growth (annual %),"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",People_practicing_open_defecation_percent_of_population_2017,Covid_Deaths_as_of_2_16_2021,Covid_Cases_as_of_2_16_2021,Covid_Recoviers_as_of_2_16_2021
0,Afghanistan,98.71320343,29.5593488,1125367,1.61,750,64.486,NaN,2.3847611,0.21,12.74018624,2428,55518,48514
1,Albania,100,0,303137,30.64,77.3,78.458,NaN,-0.246732042,7.17,0,1582,94651,58794
2,Algeria,100,28.28048,6442442,9.54,408.8,76.693,1.5477,2.00737383,0.95,0.772990468,2945,111069,76330
3,Andorra,100,NaN,NaN,NaN,NaN,NaN,NaN,0.006493211,11.02,0,107,10555,10022
4,Angola,43.25925827,78.15964,1516628,18.92,1030,60.782,0.4075,3.276133875,6.94,19.9207369,494,20389,18929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,Vietnam,100,762.66195,47049671,25.32,373,75.317,NaN,0.992716375,8.66,2.989243724,35,2311,1573
195,West Bank and Gaza,100,NaN,NaN,24.74,50,73.895,NaN,2.533011484,NaN,0.163597726,1942,169512,157982
196,"Yemen, Rep.",62,3.27,336310,NaN,NaN,66.096,0.7852,2.357022518,0.051,19.57347909,618,2148,1431
197,Zambia,39.81262207,75.07739773,16633.13894,11.55,380,63.51,1.3376,2.912801004,6.54,19.302868,974,70823,63609


In [38]:
World_Bank_Covid_Data_Cleaned = World_Bank_Covid_Data_Combined.astype({
    'Country_Name': 'category', 
    'Covid_Deaths_as_of_2_16_2021': 'int64',
    'Covid_Cases_as_of_2_16_2021': 'int64',
    'Covid_Recoviers_as_of_2_16_2021': 'int64',
    'Access to electricity (% of population)': 'float64',
    'Air transport, freight (million ton-km)': 'float64',
    'Air transport, passengers carried': 'float64',
    'Automated teller machines (ATMs) (per 100,000 adults)' : 'float64',
    'Cost to import, border compliance (US$)' : 'float64',
    'Life expectancy at birth, total (years)' : 'float64',
    'Nurses and midwives (per 1,000 people)' : 'float64',
    'Population growth (annual %)' : 'float64',
    'People_practicing_open_defecation_percent_of_population_2017' : 'float64',
    'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)' : 'float64'
        })

World_Bank_Covid_Data_Cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 0 to 198
Data columns (total 14 columns):
 #   Column                                                                                                Non-Null Count  Dtype   
---  ------                                                                                                --------------  -----   
 0   Country_Name                                                                                          199 non-null    category
 1   Access to electricity (% of population)                                                               198 non-null    float64 
 2   Air transport, freight (million ton-km)                                                               154 non-null    float64 
 3   Air transport, passengers carried                                                                     153 non-null    float64 
 4   Automated teller machines (ATMs) (per 100,000 adults)                                             

In [39]:
World_Bank_Covid_Data_Cleaned.to_csv("./World_Bank_Covid_Data_Cleaned.csv")



In our final .csv output, Null will be viewed as blank, whereas zeros will be left as zero. This would allow it to easily be imported into any future frameworks easily enough. Keep in mind that some countries really do have zero for categories that may otherwise seem like they should have data, such as air freight for example. Furthermore, some countires claim percentages at zero that may in reality be simply very close to zero. Many of these variables are only estimations.